In [ ]:
import sys
sys.path.insert(1, 'c:\\Users\\abbaj\\Desktop\\Research\\rcnn')

In [ ]:
import torch

from src.data_loaders.data_loader import Data_Loader
from src.models.faster_rcnn import Faster_RCNN

from PIL import Image

from pytorch_grad_cam import EigenCAM
from pytorch_grad_cam.ablation_layer import AblationLayerFasterRCNN
from pytorch_grad_cam.utils.model_targets import FasterRCNNBoxScoreTarget
from pytorch_grad_cam.utils.reshape_transforms import fasterrcnn_reshape_transform
from pytorch_grad_cam.utils.image import show_cam_on_image, scale_accross_batch_and_channels, scale_cam_image

from torchvision.transforms import ConvertImageDtype, ToTensor
from torchvision.transforms.functional import to_pil_image
from torchvision.utils import draw_bounding_boxes

import cv2
import numpy as np

import glob2

In [ ]:
imagesPath = "C:/Users/abbaj/Desktop/Research/IDCIAv2/images/"
annotationsPath = "C:/Users/abbaj/Desktop/Research/IDCIAv2/bounding_boxes_faster_rcnn_larger/"
loadFile = "c:/Users/abbaj/Desktop/Research/rcnn/src/weights/faster_rcnn/Weights(6,199)_31-03-24-20-19-26.pt"

data_loader = Data_Loader(
    images_path=imagesPath,
    annotations_path=annotationsPath,
    batch_size=1,
)

rcnn = Faster_RCNN()
rcnn.load_model(model_path=loadFile)

In [ ]:
image_idx = 249

rcnn.model.eval()
image_files: list = glob2.glob(imagesPath + "*.tiff")
image = ToTensor()(Image.open(image_files[image_idx]))
outputs = rcnn.model([image])
print(f"Image: {image_files[image_idx]} loaded.")
input_tensor_img = ConvertImageDtype(dtype=torch.uint8)(image)
box = draw_bounding_boxes(
    input_tensor_img,
    boxes=outputs[0]["boxes"],
    width=3,
    colors=["red"] * len(outputs[0]["boxes"]),
)
im = to_pil_image(box.detach())
im.show()

In [ ]:
def draw_boxes(boxes, image):
    for i, box in enumerate(boxes):
        cv2.rectangle(
            image,
            (int(box[0]), int(box[1])),
            (int(box[2]), int(box[3])),
            (255,0,0), 2
        )
        cv2.putText(image, 'DAPI', (int(box[0]), int(box[1] - 5)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 2,
                    lineType=cv2.LINE_AA)
    return image

In [ ]:
boxes, classes, labels = [], [], []
for index in range(len(outputs)):
    boxes.append(outputs[index]["boxes"].detach().numpy())
    classes.append(outputs[index]["labels"])
    labels.append(outputs[index]["labels"])
boxes = np.int32(boxes)[0,:,:]
classes = classes[0].detach().numpy()
labels = labels[0].detach().numpy()

In [ ]:
input_tensor_img
torch.unsqueeze(input_tensor_img, 0) / 255

In [ ]:
target_layers = [rcnn.model.backbone]
targets = [FasterRCNNBoxScoreTarget(labels=labels, bounding_boxes=boxes)]
cam = EigenCAM(rcnn.model,
               target_layers,
               reshape_transform=fasterrcnn_reshape_transform)

grayscale_cam = cam(torch.unsqueeze(input_tensor_img, 0) / 255, targets=targets)

In [ ]:
def to_rgb(im):
    w, h = im.shape
    ret = np.empty((w, h, 3), dtype=np.uint8)
    ret[:, :, 0] = im
    ret[:, :, 1] = im
    ret[:, :, 2] = im
    return ret

In [ ]:
# Take the first image in the batch:
# grayscale_cam = grayscale_cam[0, :]
image_float_np = np.float32(input_tensor_img) / 255
print(to_rgb(image_float_np[0,:,:]).shape)
print(grayscale_cam[0,:,:].shape)
cam_image = show_cam_on_image(to_rgb(image_float_np[0,:,:]), grayscale_cam[0,:,:], use_rgb=True)
# And lets draw the boxes again:
image_with_bounding_boxes = draw_boxes(boxes, cam_image)
Image.fromarray(image_with_bounding_boxes)